In [1]:
from fast_ml.model_development import train_valid_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import vtreat
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

np.random.seed(7)

d:\Python\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Python\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Python\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
TRAIN_RADIF_PATH = "../besar/exported_train_submission4_radif.csv"
TEST_RADIF_PATH = "../besar/exported_test_submission4_radif.csv"
TRAIN_PATH = "../../datasets/train.csv"
TEST_PATH = "../../datasets/test.csv"
SAMPLE_SUBMISSION_PATH = "../../datasets/sample_submission.csv"

In [3]:
train = pd.read_csv(TRAIN_RADIF_PATH)
test = pd.read_csv(TEST_RADIF_PATH)
train.head()

,rain_1h,rain_3h_is_bad,snow_1h_is_bad,snow_3h_is_bad,hum,wind_spd,prssr,weekday_sin,wind_deg,weekday_cos,...,grnd_level_impact_code,grnd_level_deviation_code,grnd_level_prevalence_code,grnd_level_lev_nan,grnd_level_lev_undetermined,sea_level_impact_code,sea_level_deviation_code,sea_level_prevalence_code,sea_level_lev_undetermined,sea_level_lev_nan
0,0.00,0.0,1.0,1.0,95.0,0.82,1012.0,0.0,320.0,1.0,...,0.000434,0.831858,0.435822,1.0,0.0,0.000846,0.830729,0.451970,1.0,0.0
1,0.00,0.0,0.0,0.0,95.0,0.96,1012.0,0.0,338.0,1.0,...,0.001181,0.833567,0.435822,1.0,0.0,-0.001780,0.828941,0.435747,0.0,1.0
2,0.00,0.0,1.0,1.0,86.0,1.22,1012.0,0.0,339.0,1.0,...,-0.000455,0.833128,0.435257,0.0,1.0,-0.001013,0.831249,0.435747,0.0,1.0
3,0.13,0.0,0.0,1.0,84.0,1.08,1012.0,0.0,342.0,1.0,...,-0.000238,0.830249,0.435257,0.0,1.0,-0.000689,0.830050,0.435747,0.0,1.0
4,0.34,0.0,1.0,0.0,87.0,0.86,1011.0,0.0,336.0,1.0,...,-0.000238,0.830249,0.435257,0.0,1.0,-0.000689,0.830050,0.435747,0.0,1.0


In [4]:
num_col_aneh = ["prssr", "wind_deg", "max_temp", "feels",
                "hum", "d_point", "temp", "min_temp", "wind_spd"]
train[num_col_aneh].describe().T[["min", "max"]]

,min,max
prssr,1001.00,5075.27
wind_deg,0.00,360.00
max_temp,22.37,132.59
feels,22.35,131.79
hum,29.00,437.61
d_point,14.49,121.27
temp,21.55,127.31
min_temp,14.12,123.54
wind_spd,0.00,5.08


In [5]:
test[num_col_aneh].describe().T[["min", "max"]]

,min,max
prssr,-100.00,1014.00
wind_deg,0.00,360.00
max_temp,-273.15,33.76
feels,-10005.49,39.49
hum,0.00,99.00
d_point,19.51,27.14
temp,-9998.49,32.68
min_temp,-9998.97,31.04
wind_spd,0.00,3.62


In [6]:
MAX_TEMP = 40
MAX_HUM = 120
MAX_MAX_TEMP = 40
MAX_FEELS = 50
MAX_PRESSURE = 1100
MAX_MIN_TEMP = 40
MAX_DPOINT = 30

transformed_train = train.copy()

print(transformed_train.shape)
transformed_train = transformed_train[transformed_train["max_temp"] < MAX_MAX_TEMP]
transformed_train = transformed_train[transformed_train["hum"] < MAX_HUM]
transformed_train = transformed_train[transformed_train["feels"] < MAX_FEELS]
transformed_train = transformed_train[transformed_train["prssr"] < MAX_PRESSURE]
transformed_train = transformed_train[transformed_train["min_temp"] < MAX_MIN_TEMP]
transformed_train = transformed_train[transformed_train["temp"] < MAX_TEMP]
print(transformed_train.shape)

(336438, 156)
(307319, 156)


In [7]:
transformed_test = test.copy()

transformed_test.iloc[48707].loc[num_col_aneh] = [1010., 170., 30., 30., 0., 27.14, 30., 15., 2.41]
pd.DataFrame(transformed_test.iloc[48707]).loc[num_col_aneh]

,48707
prssr,1010.00
wind_deg,170.00
max_temp,30.00
feels,30.00
hum,0.00
d_point,27.14
temp,30.00
min_temp,15.00
wind_spd,2.41


In [8]:
ori_train = pd.read_csv(TRAIN_PATH)
ori_test = pd.read_csv(TEST_PATH)

ori_train["datetime_iso"] = pd.to_datetime(ori_train['datetime_iso'])
ori_test["datetime_iso"] = pd.to_datetime(ori_test['datetime_iso'])
transformed_train["hour"] = ori_train['datetime_iso'].dt.hour
transformed_test["hour"] = ori_test['datetime_iso'].dt.hour

In [9]:
def hour_to_time(x):
    if x >= 0 and x < 6:
        return "midnight"
    elif x >= 6 and x < 12:
        return "morning"
    elif x >= 12 and x < 18:
        return "afternoon"
    else:
        return "evening"


transformed_train["hour"] = transformed_train["hour"].apply(
    lambda x: hour_to_time(x))
transformed_test["hour"] = transformed_test["hour"].apply(
    lambda x: hour_to_time(x))

In [10]:
ohe_hour_train = pd.get_dummies(transformed_train["hour"], dtype=float)
ohe_hour_test = pd.get_dummies(transformed_test["hour"], dtype=float)

transformed_train = pd.concat([transformed_train, ohe_hour_train], axis=1)
transformed_test = pd.concat([transformed_test, ohe_hour_test], axis=1)

transformed_train.drop(["hour"], axis=1, inplace=True)
transformed_test.drop(["hour"], axis=1, inplace=True)

In [11]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(
    transformed_train,
    target='rain_1h',
    train_size=0.7, valid_size=0.15, test_size=0.15, random_state=7)


In [106]:
from pycaret.regression import *

s = setup(
    data=transformed_train,
    target='rain_1h',
    use_gpu=True
)

best_model = compare_models(
    sort='RMSE',
    include=['catboost', 'xgboost', 'lightgbm']
)

,Description,Value
0,Session id,4576
1,Target,rain_1h
2,Target type,Regression
3,Original data shape,"(307319, 160)"
4,Transformed data shape,"(307319, 160)"
5,Transformed train set shape,"(215123, 160)"
6,Transformed test set shape,"(92196, 160)"
7,Numeric features,159
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.3625,0.4778,0.6912,0.3084,0.3151,1.1880,7.4530
lightgbm,Light Gradient Boosting Machine,0.3635,0.4835,0.6953,0.3003,0.3165,1.1609,2.8710
xgboost,Extreme Gradient Boosting,0.3676,0.4886,0.6990,0.2929,0.3204,1.2309,2.5160


In [12]:
# from umap import UMAP

# umapmap = UMAP(n_components=10, random_state=7)

# umapmap_train = umapmap.fit_transform(transformed_train.drop(["rain_1h"], axis=1))

In [23]:
from pycaret.regression import *

s = setup(
    data=pd.concat([
                    # pd.DataFrame(umapmap_train),
                    transformed_train.reset_index(drop=True),
                    # pd.DataFrame(pca_train, columns=[
                    #         "pca_" + str(i) for i in range(50)])
                    ], axis=1),
    target='rain_1h',
    use_gpu=True
)

best_model = compare_models(
    sort='RMSE',
    include=['catboost'],
)

,Description,Value
0,Session id,357
1,Target,rain_1h
2,Target type,Regression
3,Original data shape,"(307319, 160)"
4,Transformed data shape,"(307319, 160)"
5,Transformed train set shape,"(215123, 160)"
6,Transformed test set shape,"(92196, 160)"
7,Numeric features,159
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.3625,0.4762,0.6900,0.3059,0.3152,1.1829,7.4180


In [24]:
tuned_best_model = tune_model(best_model, optimize='RMSE', )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3685,0.4921,0.7015,0.2869,0.3190,1.1657
1,0.3662,0.4822,0.6944,0.2917,0.3179,1.1637
2,0.3671,0.4825,0.6946,0.3054,0.3159,1.1325
3,0.3693,0.4905,0.7004,0.2931,0.3192,1.1815
4,0.3697,0.4978,0.7056,0.3007,0.3197,1.1786
5,0.3677,0.4869,0.6978,0.2803,0.3195,1.1724
6,0.3636,0.4739,0.6884,0.3138,0.3133,1.1790
7,0.3652,0.4720,0.6870,0.3098,0.3157,1.1803
8,0.3692,0.4852,0.6965,0.2951,0.3187,1.2048


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [25]:
final_model = finalize_model(tuned_best_model)

In [29]:
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission["rain_1h"] = final_model.predict(transformed_test)
submission

,datetime_iso,rain_1h
0,2018-01-01 00:00:00+00:00,-0.194284
1,2018-01-01 01:00:00+00:00,-0.049848
2,2018-01-01 02:00:00+00:00,0.088995
3,2018-01-01 03:00:00+00:00,0.253622
4,2018-01-01 04:00:00+00:00,0.006958
...,...,...
49363,2023-08-19 19:00:00+00:00,0.040941
49364,2023-08-19 20:00:00+00:00,0.036847
49365,2023-08-19 21:00:00+00:00,0.026180
49366,2023-08-19 22:00:00+00:00,0.017428


In [32]:
submission["rain_1h"] = submission["rain_1h"].apply(lambda x: 0. if x < 0 else x)
submission.to_csv("../submission/submission_5.csv", index=False)

In [35]:
import pickle

pickle.dump(final_model, open("../model/submission_5.pkl", "wb"))

In [41]:
transformed_train.to_parquet("../cleaned/submission_5/train.parquet", index=False)
transformed_test.to_parquet("../cleaned/submission_5/test.parquet", index=False)